<a href="https://colab.research.google.com/github/kaushal27120/My_Projects/blob/main/Spam_mail_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

fake = pd.read_csv('/content/Fake.csv')
real = pd.read_csv('/content/True.csv')

fake['label'] = 0
real['label'] = 1

df = pd.concat([fake, real]).sample(frac=1).reset_index(drop=True)
df['text'] = df['title'] + ' ' + df['text']

df = df.sample(n=2000, random_state=42).reset_index(drop=True)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {k: torch.tensor(v[idx]) for k, v in self.encodings.items()} | {'labels': torch.tensor(self.labels[idx])}
    def __len__(self):
        return len(self.labels)

train_dataset = FakeNewsDataset(train_encodings, list(y_train))
test_dataset = FakeNewsDataset(test_encodings, list(y_test))


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1,
    do_train=True,
    do_eval=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=DataCollatorWithPadding(tokenizer)
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kjasani1 (kjasani1-akademia-finans-w-i-biznesu-vistula) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.570800
20,0.233000
30,0.157700
40,0.058800
50,0.097900
60,0.005000
70,0.141100
80,0.002600
90,0.146700
100,0.003600


TrainOutput(global_step=200, training_loss=0.07435299409320578, metrics={'train_runtime': 158.7857, 'train_samples_per_second': 10.076, 'train_steps_per_second': 1.26, 'total_flos': 420977688576000.0, 'train_loss': 0.07435299409320578, 'epoch': 1.0})

In [ ]:
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(-1)

print(classification_report(y_test, pred_labels))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       211
           1       1.00      1.00      1.00       189

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, pred_labels)
print("Accuracy:", round(acc * 100, 2), "%")

Accuracy: 100.0 %


In [ ]:
print("\nFirst 10 Predictions vs Actual Labels:")
for i in range(10):
    print(f"{i+1}. Predicted: {pred_labels[i]}, Actual: {list(y_test)[i]}")


First 10 Predictions vs Actual Labels:
1. Predicted: 0, Actual: 0
2. Predicted: 0, Actual: 0
3. Predicted: 0, Actual: 0
4. Predicted: 1, Actual: 1
5. Predicted: 1, Actual: 1
6. Predicted: 1, Actual: 1
7. Predicted: 1, Actual: 1
8. Predicted: 0, Actual: 0
9. Predicted: 1, Actual: 1
10. Predicted: 0, Actual: 0
